In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing, linear_model
import os
os.chdir('C:/Users/n0269042/Documents/numerai')

import h2o
from h2o.automl import H2OAutoML

h2o.init()

training_data = pd.read_csv('data/numerai_20180430/numerai_training_data.csv', header=0)
prediction_data = pd.read_csv('data/numerai_20180430/numerai_tournament_data.csv', header=0)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,4 days 1 hour 17 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.4
H2O cluster version age:,1 month and 22 days
H2O cluster name:,H2O_from_python_n0269042_c1dwk2
H2O cluster total nodes:,1
H2O cluster free memory:,20.83 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [7]:
train = training_data[training_data.era.isin(['era'+str(i+1) for i in range(100)])].drop(['id', 'era', 'data_type'], axis=1)
leaderboard = training_data[training_data.era.isin(['era'+str(i+101) for i in range(20)])].drop(['id', 'era', 'data_type'], axis=1)
train = h2o.H2OFrame(train)
leaderboard = h2o.H2OFrame(leaderboard)

validation = prediction_data[prediction_data.data_type == 'validation'].drop(['id', 'era', 'data_type'], axis=1)
validation = h2o.H2OFrame(validation)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
x = train.columns
y = "target"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
leaderboard[y] = leaderboard[y].asfactor()

# Run AutoML for 30 seconds
aml = H2OAutoML(max_runtime_secs = 18000)
aml.train(x = x, y = y,
          training_frame = train,
          leaderboard_frame = leaderboard)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


model_id,auc,logloss
DeepLearning_grid_0_AutoML_20180501_154107_model_5,0.514306,0.694773
GBM_grid_0_AutoML_20180501_154107_model_5,0.50973,0.693047
DeepLearning_grid_0_AutoML_20180501_154107_model_1,0.509685,0.697297
GBM_grid_0_AutoML_20180501_154107_model_6,0.509497,0.693056
StackedEnsemble_AllModels_0_AutoML_20180501_154107,0.508821,0.693766
DeepLearning_grid_0_AutoML_20180501_154107_model_3,0.508357,0.698362
StackedEnsemble_BestOfFamily_0_AutoML_20180501_154107,0.507475,0.693634
GBM_grid_0_AutoML_20180501_154107_model_0,0.507138,0.693634
DeepLearning_grid_0_AutoML_20180501_154107_model_4,0.506975,0.699131
DeepLearning_grid_0_AutoML_20180501_154107_model_7,0.506786,0.693207


In [5]:
holdout = prediction_data.drop(['id', 'era', 'data_type'], axis=1)
holdout = h2o.H2OFrame(holdout)
preds = aml.predict(holdout)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


In [15]:
test_submission = pd.DataFrame({'id': prediction_data.id, 'probability': preds.as_data_frame().p1})
test_submission.to_csv('submission/test_submission_20180430_1.csv', index=False)

predictions must be between 0.3 and 0.7

In [16]:
test_submission.probability[test_submission.probability > 0.7] = 0.7
test_submission.probability[test_submission.probability < 0.3] = 0.3

test_submission.to_csv('submission/test_submission_20180430_2.csv', index=False)

C:\Users\n0269042\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\n0269042\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
